# **Advanced SQL Queries**

In [30]:
from pyspark.sql import SparkSession
import sqlite3
import pandas as pd

# 1. Start Spark session
spark = SparkSession.builder \
    .appName("ProductAprioriAnalysis") \
    .getOrCreate()

# 2. Connecting to DB
db_path = "/content/product_apriori.db"
conn = sqlite3.connect(db_path)

# 3. Getting only non-internal tables
tables_df = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';",
    conn
)

print("Tables in DB:")
print(tables_df)

table_names = tables_df['name'].tolist()
spark_tables = {}

for tbl in table_names:
    pdf = pd.read_sql_query(f"SELECT * FROM {tbl};", conn)

    if pdf.empty:
        print(f"Skipping empty table: {tbl}")
        continue

    # Dropping columns that are completely NULL (all NaN)
    null_cols = [c for c in pdf.columns if pdf[c].isna().all()]
    if null_cols:
        print(f"Table {tbl}: dropping all-null columns: {null_cols}")
        pdf = pdf.drop(columns=null_cols)

    # Now Spark can infer types safely
    sdf = spark.createDataFrame(pdf)
    sdf.createOrReplaceTempView(tbl)
    spark_tables[tbl] = sdf
    print(f"Loaded table: {tbl}, rows: {sdf.count()}")


Tables in DB:
              name
0         customer
1          product
2  product_details
3          ratings
4          reviews
5     transactions
6    market_basket
7     top_products
Loaded table: customer, rows: 552
Loaded table: product, rows: 65
Table product_details: dropping all-null columns: ['color']
Loaded table: product_details, rows: 65
Loaded table: ratings, rows: 1604
Loaded table: reviews, rows: 4166
Loaded table: transactions, rows: 38765
Loaded table: market_basket, rows: 1238
Loaded table: top_products, rows: 279


In [31]:
spark.sql("SELECT * FROM ratings LIMIT 5").show(truncate=False)
spark.sql("SELECT * FROM reviews LIMIT 5").show(truncate=False)
spark.sql("SELECT * FROM product_details LIMIT 5").show(truncate=False)


+-------+-----------+--------------+-------------+
|item_id|parent_asin|average_rating|rating_number|
+-------+-----------+--------------+-------------+
|26     |B09PF8SQT7 |2.8           |8            |
|3      |B07YCVLMCX |3.1           |47           |
|64     |B07H57T9PV |4.0           |77           |
|45     |B07X3QVZQ9 |4.2           |21           |
|13     |B09HWSX63J |3.2           |2            |
+-------+-----------+--------------+-------------+

+------+--------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------+------------------

# **Query 1:**
# **Most Frequently Reviewed & Highly Rated Products**

In [32]:
spark.sql("""
WITH title_counts AS (
    SELECT
        r.parent_asin,
        pd.title,
        COUNT(*) AS title_count
    FROM ratings r
    JOIN product_details pd
        ON pd.item_id = r.item_id
    GROUP BY r.parent_asin, pd.title
),
best_title AS (
    SELECT parent_asin, title
    FROM (
        SELECT
            parent_asin,
            title,
            title_count,
            ROW_NUMBER() OVER (PARTITION BY parent_asin ORDER BY title_count DESC) AS rn
        FROM title_counts
    )
    WHERE rn = 1
)

SELECT
    r.parent_asin,
    bt.title,
    COUNT(rv.rating) AS review_count,
    r.average_rating,
    r.rating_number
FROM ratings r
JOIN reviews rv
    ON rv.parent_asin = r.parent_asin
JOIN best_title bt
    ON bt.parent_asin = r.parent_asin
GROUP BY
    r.parent_asin, bt.title, r.average_rating, r.rating_number
ORDER BY
    review_count DESC,
    average_rating DESC
LIMIT 10
""").show(truncate=False)


+-----------+-----------------------------------------------------------------------------------------------------------------------------------------+------------+--------------+-------------+
|parent_asin|title                                                                                                                                    |review_count|average_rating|rating_number|
+-----------+-----------------------------------------------------------------------------------------------------------------------------------------+------------+--------------+-------------+
|B01FW4ZCJM |Hershey's Reese's Crunchy Cookie Peanut butter/Chocolate Candy 1.4 oz.                                                                   |4           |4.6           |2104         |
|B0C7RJ16K2 |TORTUGA Caribbean Coconut Rum Cake - 32 oz Rum Cake - The Perfect Premium Gourmet Gift for Gift Baskets, Parties, Holidays, and Birthdays|3           |4.5           |1180         |
|B09HCFRWNP |Dark Roast Pure C

Explanation:
This report identifies the products that have attracted the most reviews and shows their average star ratings, highlighting which items are both popular and well-rated.

# **Query 2:**
# **Highest-Rated Products with Minimum Number of Ratings**

In [33]:
spark.sql("""SELECT
    r.parent_asin,
    pd.title,
    r.average_rating,
    r.rating_number
FROM ratings r
JOIN product_details pd
    ON pd.item_id = r.item_id
WHERE r.rating_number >= 20          -- filter: at least 20 ratings
ORDER BY
    r.average_rating DESC,
    r.rating_number DESC
LIMIT 10;
""").show(truncate=False)

+-----------+----------------------------------------------------------------------------------------------------------------------+--------------+-------------+
|parent_asin|title                                                                                                                 |average_rating|rating_number|
+-----------+----------------------------------------------------------------------------------------------------------------------+--------------+-------------+
|B07HKWV1R2 |Cherry Sours Chewy Candy Balls - 3 lbs of Tart Fresh Delicious Bulk Candy                                             |5.0           |20           |
|B007SAUB36 |Del Monte Garlic Pasta Sauce, 24 Ounce (Pack of 12)                                                                   |5.0           |20           |
|B0748MWKNM |Handlmaier's Sweet Bavarian Mustard-8 oz - 230 g- IMPORTED- Shipping from USA                                         |4.9           |274          |
|B07PGK25N5 |Handlmaier's Sw

Explaination: This query filters products with at least 20 ratings and highlights the top 10 items with the highest average ratings, revealing that condiments, seasonings, and pantry staples dominate the list with near-perfect customer satisfaction.

# **Query 3:**
# **Average Sentiment Score per Product Category**

In [34]:
spark.sql("""
SELECT
    category,
    AVG(avg_sentiment)   AS avg_sentiment,
    AVG(average_rating)  AS avg_rating,
    SUM(review_count)    AS total_reviews,
    COUNT(*)             AS num_products
FROM top_products
GROUP BY category
ORDER BY total_reviews DESC
""").show(truncate=False)


+--------------------------------------+-------------------+------------------+-------------+------------+
|category                              |avg_sentiment      |avg_rating        |total_reviews|num_products|
+--------------------------------------+-------------------+------------------+-------------+------------+
|Cookies                               |0.4284             |4.420000000000001 |42.0         |15          |
|Nut & Seed Butters                    |0.498              |4.6               |36.0         |9           |
|Cereals                               |1.0                |4.6               |30.0         |15          |
|Soups, Stocks & Broths                |1.0                |4.5               |30.0         |15          |
|Chocolate                             |0.6010000000000002 |4.16              |27.0         |15          |
|Jams, Jellies & Sweet Spreads         |0.8320000000000001 |4.466666666666667 |21.0         |9           |
|Candy & Chocolate                   

Explanation:
This report uses the top_products table to summarize average sentiment and ratings by product category.
It helps you answer: “Which categories are loved the most (high sentiment + rating) and have enough review volume to be reliable?”

GROUP BY category – aggregates all products in each category.

AVG(avg_sentiment) – average sentiment score of the products in that category.

AVG(average_rating) – average star rating.

SUM(review_count) – total number of reviews across all products in that category.

COUNT(*) – how many products you have in that category.

# **Query 4:**
# **Monthly Trends in Product Ratings**

In [35]:
spark.sql("""
SELECT
    date_format(from_unixtime(timestamp / 1000), 'yyyy-MM') AS year_month,
    COUNT(*)                                              AS review_count,
    AVG(rating)                                           AS avg_rating
FROM reviews
GROUP BY year_month
ORDER BY year_month
""").show(truncate=False)


+----------+------------+------------------+
|year_month|review_count|avg_rating        |
+----------+------------+------------------+
|2006-07   |1           |5.0               |
|2007-08   |1           |5.0               |
|2009-05   |1           |4.0               |
|2009-11   |1           |5.0               |
|2010-09   |1           |5.0               |
|2010-11   |1           |5.0               |
|2011-01   |2           |5.0               |
|2011-08   |1           |5.0               |
|2011-09   |1           |3.0               |
|2012-02   |1           |5.0               |
|2012-08   |1           |3.0               |
|2012-12   |1           |5.0               |
|2013-01   |3           |3.6666666666666665|
|2013-02   |1           |5.0               |
|2013-04   |4           |4.0               |
|2013-05   |9           |4.555555555555555 |
|2013-06   |3           |5.0               |
|2013-07   |1           |5.0               |
|2013-08   |2           |5.0               |
|2013-09  

Explanation
This query shows how average ratings and review volume change month by month.
You can see if ratings are trending up or down over time, and which months are more active in terms of reviews.

# **Query 5:**
# **Relationship Between Review Length and Rating**

In [36]:
spark.sql("""
WITH reviews_with_len AS (
    SELECT
        rating,
        LENGTH(text) AS review_len
    FROM reviews
    WHERE text IS NOT NULL
),
bucketed AS (
    SELECT
        CASE
            WHEN review_len < 100 THEN 'short (<100 chars)'
            WHEN review_len BETWEEN 100 AND 300 THEN 'medium (100–300 chars)'
            WHEN review_len BETWEEN 301 AND 600 THEN 'long (301–600 chars)'
            ELSE 'very long (>600 chars)'
        END AS length_bucket,
        rating
    FROM reviews_with_len
)
SELECT
    length_bucket,
    COUNT(*)          AS num_reviews,
    AVG(rating)       AS avg_rating
FROM bucketed
GROUP BY length_bucket
ORDER BY
    CASE length_bucket
        WHEN 'short (<100 chars)'          THEN 1
        WHEN 'medium (100–300 chars)'      THEN 2
        WHEN 'long (301–600 chars)'        THEN 3
        WHEN 'very long (>600 chars)'      THEN 4
    END
""").show(truncate=False)


+----------------------+-----------+------------------+
|length_bucket         |num_reviews|avg_rating        |
+----------------------+-----------+------------------+
|short (<100 chars)    |1849       |4.3791238507301244|
|medium (100–300 chars)|1511       |4.246194573130377 |
|long (301–600 chars)  |551        |4.288566243194192 |
|very long (>600 chars)|255        |4.1607843137254905|
+----------------------+-----------+------------------+



Explanation
This analyzes the relationship between review length and rating by grouping into length buckets.
You can see, for example, whether longer reviews tend to be more negative or more positive on average.

# **Query 6:**
# **Top Keywords in Positive vs Negative Reviews**

In [37]:
spark.sql("""
WITH labelled_reviews AS (
    SELECT
        CASE
            WHEN rating >= 4 THEN 'positive'
            WHEN rating <= 2 THEN 'negative'
            ELSE 'neutral'
        END AS sentiment,
        text
    FROM reviews
    WHERE text IS NOT NULL
),
keyword_counts AS (
    SELECT sentiment, 'delicious' AS keyword, COUNT(*) AS occurrences
    FROM labelled_reviews
    WHERE lower(text) LIKE '%delicious%'
    GROUP BY sentiment

    UNION ALL

    SELECT sentiment, 'tasty' AS keyword, COUNT(*) AS occurrences
    FROM labelled_reviews
    WHERE lower(text) LIKE '%tasty%'
    GROUP BY sentiment

    UNION ALL

    SELECT sentiment, 'bad' AS keyword, COUNT(*) AS occurrences
    FROM labelled_reviews
    WHERE lower(text) LIKE '%bad%'
    GROUP BY sentiment

    UNION ALL

    SELECT sentiment, 'disappointed' AS keyword, COUNT(*) AS occurrences
    FROM labelled_reviews
    WHERE lower(text) LIKE '%disappointed%'
    GROUP BY sentiment
)
SELECT *
FROM keyword_counts
ORDER BY keyword, sentiment
""").show(truncate=False)


+---------+------------+-----------+
|sentiment|keyword     |occurrences|
+---------+------------+-----------+
|negative |bad         |29         |
|neutral  |bad         |11         |
|positive |bad         |49         |
|negative |delicious   |8          |
|neutral  |delicious   |7          |
|positive |delicious   |446        |
|negative |disappointed|48         |
|neutral  |disappointed|17         |
|positive |disappointed|38         |
|negative |tasty       |9          |
|neutral  |tasty       |8          |
|positive |tasty       |306        |
+---------+------------+-----------+



Explanation
This query compares how often certain keywords appear in positive vs negative reviews.
You can quickly see which words are associated more with happy customers vs unhappy customers.

# **Query 7:**
# **Customer Segments by Sentiment Polarity**

In [38]:
spark.sql("""
WITH customer_stats AS (
    SELECT
        rv.user_id,
        COUNT(*)          AS num_reviews,
        AVG(rv.rating)    AS avg_rating
    FROM reviews rv
    GROUP BY rv.user_id
),
segmented AS (
    SELECT
        cs.user_id,
        cs.num_reviews,
        cs.avg_rating,
        CASE
            WHEN cs.avg_rating >= 4 THEN 'Positive'
            WHEN cs.avg_rating BETWEEN 2.5 AND 4 THEN 'Neutral'
            ELSE 'Negative'
        END AS sentiment_segment
    FROM customer_stats cs
)
SELECT
    s.sentiment_segment,
    COUNT(*)                     AS num_customers,
    AVG(s.num_reviews)           AS avg_reviews_per_customer
FROM segmented s
GROUP BY s.sentiment_segment
ORDER BY num_customers DESC
""").show(truncate=False)


+-----------------+-------------+------------------------+
|sentiment_segment|num_customers|avg_reviews_per_customer|
+-----------------+-------------+------------------------+
|Positive         |400          |8.1925                  |
|Neutral          |115          |6.965217391304348       |
|Negative         |37           |2.3783783783783785      |
+-----------------+-------------+------------------------+



Explanation
This report segments customers based on their average rating into Positive/Neutral/Negative.
It shows how many customers fall into each sentiment group and how active they are (average number of reviews).

# **Query 8:**
# **Sentiment Comparison Between Product Categories**

In [39]:
spark.sql("""
SELECT
    category,
    AVG(avg_sentiment)   AS avg_sentiment,
    AVG(average_rating)  AS avg_rating,
    SUM(review_count)    AS total_reviews,
    COUNT(*)             AS num_products
FROM top_products
GROUP BY category
ORDER BY total_reviews DESC
""").show(truncate=False)


+--------------------------------------+-------------------+------------------+-------------+------------+
|category                              |avg_sentiment      |avg_rating        |total_reviews|num_products|
+--------------------------------------+-------------------+------------------+-------------+------------+
|Cookies                               |0.4284             |4.420000000000001 |42.0         |15          |
|Nut & Seed Butters                    |0.498              |4.6               |36.0         |9           |
|Cereals                               |1.0                |4.6               |30.0         |15          |
|Soups, Stocks & Broths                |1.0                |4.5               |30.0         |15          |
|Chocolate                             |0.6010000000000002 |4.16              |27.0         |15          |
|Jams, Jellies & Sweet Spreads         |0.8320000000000001 |4.466666666666667 |21.0         |9           |
|Candy & Chocolate                   

This compares categories against each other in terms of:

Average sentiment score

Average star rating

Total review volume

Number of products

It helps you see which categories are strongest overall and which ones may need improvement.